In [1]:
from environment_manager import *

Create the version Environment

In [2]:

name = "240308-initial_test"
env = load_create_environment(name)
gct = GHComponentTable
GHComponentTable.initialise()



Setting environment variables
Copying vanilla components
File copied successfully from C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240307-CoreComponents\vanilla_components.csv to ExtractionEnvironments\240308-initial_test\00-VanillaComponents\vanilla_components.csv.
Copying components
Copying gh files


In [3]:
# gh_file = env.dirs["files"] /"210309_Canopy Modelling.gh"
# # gh_file = env.dirs["files"] /'test.gh'
# doc = GHProcessor.get_ghdoc(str(gh_file))
# t_obj = ["Panel"]
# illegal = ["Sketch","Scribble"]
# test_objects = []
# canvas = Canvas(gh_file.stem, doc, env)
# for object in doc.Objects:
#     c = GHComponent(object)
#     print(f"{c.name}, {c.get_connections(canvas)}")


In [4]:
import Grasshopper.Kernel as ghk
from Grasshopper.Kernel import GH_DocumentIO

In [5]:

# file = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\test\all_components_.gh"
# doc = GHProcessor.get_ghdoc(file)
# 
# comp = GHComponentTable
# comp.initialise()
# 
# object_proxies = GHComponentTable.object_proxies
# 
# 
# i_space = 100.
# j_space = 200.
# currenttype = object_proxies[0].category
# pivot = [0, 0]
# problems = [] 
# for obj in object_proxies:
#     if obj.name == "FeatureRequest":
#         continue
#     if obj.category != currenttype:
#         currenttype = obj.category
#         pivot[1] += j_space
#         pivot[0] = 0
#     try:
#         print(obj.name)
#         dobj = obj.obj_proxy.CreateInstance()
#         dobj.Attributes.Pivot = System.Drawing.PointF(pivot[0], pivot[1])
#         doc.AddObject(dobj, False)
#         pass
#     except AttributeError as e:
#         problems.append(obj)
#         print(f"{e}:{obj.name}")
#         continue
#     except Exception as e:
#         problems.append(obj)
#         print(f"{e}:{obj.name}")
#     pivot[0] += i_space
# ghdio = GH_DocumentIO(doc)
# ghdio.Save()
# # print(problems)

In [34]:
def doc_save(doc):
    ghdio = GH_DocumentIO(doc)
    ghdio.Save()    

def get_component_type(component):
    pattern = r"^(.*?)_OBSOLETE"
    match = re.search(pattern, str(component))
    return match.group(1) if match else None

def create_new_component(component_type):
    replacement_guids = GHComponentTable.get_guid_by_type(component_type)
    if replacement_guids:
        new_component_proxy = GHComponentTable.search_component_by_guid(System.Guid(replacement_guids[0]))
        if new_component_proxy:
            return new_component_proxy.CreateInstance()
    return None

def rewire_connections(old_component, new_component):
    old_component_obj = GHComponent_.convert_cls(old_component, ghk.IGH_Component)
    new_component_obj = GHComponent_.convert_cls(new_component, ghk.IGH_Component)

    if old_component_obj and new_component_obj:
        for input, new_input in zip(old_component_obj.Params.Input, new_component_obj.Params.Input):
            for source in input.Sources:
                new_input.AddSource(source)
        for output, new_output in zip(old_component_obj.Params.Output, new_component_obj.Params.Output):
            for recipient in output.Recipients:
                recipient.AddSource(new_output)

    else:
        old_param_obj = GHComponent_.convert_cls(old_component, ghk.IGH_Param)
        new_param_obj = GHComponent_.convert_cls(new_component, ghk.IGH_Param)

        if old_param_obj and new_param_obj:
            for source in old_param_obj.Sources:
                new_param_obj.AddSource(source)
            for recipient in old_param_obj.Recipients:
                recipient.AddSource(new_param_obj)

        else:
            print(f"Cannot rewire connections. Incompatible component types: {type(old_component)} and {type(new_component)}")


def replace_component(doc, old_component, new_component):
    if new_component:
        new_component.Attributes.Pivot = old_component.Attributes.Pivot
        doc.AddObject(new_component, False)
        rewire_connections(old_component, new_component)
        doc.RemoveObject(old_component, True)
        return True
    return False

def replace_obsolete_components(doc):
    for component in list(doc.Objects):
        if hasattr(component, 'Obsolete') and component.Obsolete:
            component_type = get_component_type(component)
            if component_type:
                new_component = create_new_component(component_type)
                if not replace_component(doc, component, new_component):
                    print(f"Could not replace obsolete component: {component.Name}")
    print("Replacement of obsolete components complete.")
    return doc


In [35]:
import re
file = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\TTD\missing comps\missingcomps.gh"
doc = GHProcessor.get_ghdoc(file)
doc = replace_obsolete_components(doc)
doc_save(doc)
            


Replacement of obsolete components complete.


,category,name,guid,nickname,description,type,library
0,X,Example,d541404d-b806-453e-8351-a954e824ed22,X,Example,Grasshopper.Kernel.Components.GH_DocExampleCom...,Grasshopper
1,Curve,Arc,bb59bffc-f54c-4682-9778-f6c3fe74fce3,Arc,"Create an arc defined by base plane, radius an...",CurveComponents.Component_Arc,Curve Components
2,Curve,Arc 3Pt,9fa1b081-b1c7-4a12-a163-0aa8da9ff6c4,Arc,Create an arc through three points.,CurveComponents.Component_Arc3Pt,Curve Components
3,Curve,Arc SED,9d2583dd-6cf5-497c-8c40-c9a290598396,Arc,"Create an arc defined by start point, end poin...",CurveComponents.Component_ArcSED,Curve Components
4,Curve,Bezier Span,30ce59ce-22a1-49ee-9e21-e6d16b3684a8,BzSpan,Construct a bezier span from endpoints and tan...,CurveComponents.Component_BezierSpan,Curve Components


In [9]:
file = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\TTD\missing comps\missingcomps.gh"
doc = GHProcessor.get_ghdoc(file)
replace_obsolete_components(doc)
# print(doc)
# for component in doc.Objects:
#     try:
#         a = GHComponentTable.get_guid_to_idx(str(component.ComponentGuid))
#         if a is None:
#             print(component)
#     except Exception as e:
#         print(e)

['86b28a7e-94d9-4791-8306-e13e10d5f8d5']
['86b28a7e-94d9-4791-8306-e13e10d5f8d5']


In [10]:
# file = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\TTD\bifocals\bifocals_on.gh"
# illegals_dict = {
#     "Bifocals": "aced9701-8be9-4860-bc37-7e22622afff4",
#     "Group" : "c552a431-af5b-46a9-a8a4-0fcbc27ef596"
# }
# illegals_set = set([str(v) for v in illegals_dict.values()])
# 
# def preprocess(doc, overwrite=False):
#     removables = []
#     # iterate over all the objects and find those in the set of objects we want to remove
#     for obj in doc.Objects:
#         if str(obj.ComponentGuid) in illegals_set:
#             removables.append(obj)
#     # remove all the unwanted objects
#     for obj in removables:
#         doc.RemoveObject(obj, True)
#     if overwrite:
#         ghdio = GH_DocumentIO(doc)
#         ghdio.Save()
#     print("preprocessing complete")
#     return doc
# 
# doc = GHProcessor.get_ghdoc(file)
# print(f"# before: {len(doc.Objects)}")
# doc = preprocess(doc, True)
# print(f"# after: {len(doc.Objects)}")


In [11]:
class GHNode_:
    def __init__(self, obj: ghk.IGH_DocumentObject):
        self.obj = obj
        self.category = obj.Category
        self.name = obj.Name
        self.id = str(obj.InstanceGuid)
        self.position = obj.Attributes.Pivot if hasattr(obj.Attributes, "Pivot") else None
        self.uid = f"{self.category}_{self.name}_{self.id[-5:]}"
        # Assuming global_idx is somehow related to GHComponentTable, which might need instance reference
        self.global_idx = GHComponentTable.component_to_idx(self)  # This requires GHComponentTable method adjustment
        self.graph_id = None

    def get_recipients(self):
        """To be implemented by the subclass"""
        pass

    def __str__(self):
        return f"{self.uid}"

    def __repr__(self):
        return f"<GHNode {self.__str__()}>"

    def log_properties(self):
        log = {
            f"Category: {self.category}, "
            f"Name: {self.name}, "
            f"ID: {self.id[-5:]}, "
            f"Position: {self.position}"
            f"Global: {self.global_idx}"
        }
        # This method seems intended for logging or debugging, consider how it's used and adapt accordingly.

In [12]:
class GHParam_:

    def __init__(self, obj):
        self.obj = obj
        self.parent = GHNode(ghk.IGH_DocumentObject(obj).Attributes.GetTopLevel.DocObject)
        self.name = obj.Name
        self.datamapping = int(obj.DataMapping)  # enumerator 0:none, 1:Flatten, 2:Graft
        self.pkind = obj.Kind  # the kind: floating (top level), input (parameter tied to component as input), output (parameter tied to a component as an output
        self.dataEmitter = obj.IsDataProvider  # boolean stating whether this object is able to emit data
        # self.typ = obj.Type
        self.typname = obj.TypeName  # human-readable descriptor of this parameter
        self.optional = obj.Optional  # gets whether this parameter is optional to the functioning of the component
        # logging.info(f'GHComponent {self.parent.name} Params: {self.log_properties()}')

    @property
    def recipients(self):
        # if there are no recipents to this parameter, return none
        return [rcp for rcp in self.obj.Recipients] if len(self.obj.Recipients) > 0 else None

    @property
    def sources(self):
        # if there are no recipents to this parameter, return none
        return [rcp for rcp in self.obj.Sources] if len(self.obj.Sources) > 0 else None

    @property
    def data(self):
        return self.obj.VolatileData.DataDescription(False, False)

    def log_properties(self):
        properties = (
            f"Name: {self.name}, "
            f"DataMapping: {self.datamapping}, "
            f"Kind: {self.pkind}, "
            f"DataEmitter: {self.dataEmitter}, "
            f"TypeName: {self.typname}, "
            f"Optional: {self.optional}, "
            f"Data: {self.data}, "
        )
        return properties

    def __str__(self):
        repr_obj = ghk.IGH_DocumentObject(self.obj)
        return f"param:{self.name}"

    def __repr__(self):
        return f"<GHParam {self.__str__()}>"

In [13]:
from typing import Union
class GHComponent_(GHNode_):
    """Subclass of GHNode that handles GH components that implement IGH_Component.
    Each GHComponent object should contain a list of input parameter and output parameter objects.
    These parameter objects have access to the sources and recipients of the parameter"""

    def __init__(self, obj):
        super().__init__(obj)
        self.obj: Union[IGH_Component or IGH_Param] = None
        self.iparams = []
        self.oparams = []
        self.recipients = []
        self.iparams_dict = {}
        self.oparams_dict = {}

        # Attempt to initialize parameters
        self.initialize_parameters(obj)
    
    def initialize_parameters(self, obj):
        if GHComponent_.is_cls(obj, ghk.IGH_Component):
            self.obj = IGH_Component(obj)
            self.iparams = [GHParam_(p) for p in self.obj.Params.Input]
            self.oparams = [GHParam_(p) for p in self.obj.Params.Output]
        
        elif GHComponent_.is_cls(obj, ghk.IGH_Param):
            self.obj = IGH_Param(obj)
            param = GHParam_(self.obj)
            self.iparams = [param]
            self.oparams = [param]
            
        else:
            self.obj = GHComponentTable.idx_to_component()
    
    @staticmethod
    def is_cls(obj, clas):
        try:
            clas(obj)
            return True
        except Exception:
            return False
    @staticmethod
    def convert_cls(obj, clas):
        try:
            return clas(obj)
        except Exception:
            return None
            

In [14]:
import random
n = 100
l = len(doc.Objects)
random.seed(2)
docobjs = doc.Objects
issues = []
issue_type = []
for obj in random.choices(list(docobjs), k=l):
 
    a = GHComponent_(obj)
    print(a.name)
#     except Exception as e:
#         issue_type.append(e)
#         issues.append(obj)
# # print(issues)
# print(len(issues))


Flip
Bounding Box
Panel Connections
Mesh
Line
Deconstruct Brep
Number Slider
Panel
Curve
Curve
Distance
Panel
Catenary
Sort List
Area
Flip Brep
Point In Brep
Flatten Tree
Length
Bounds
Point


TypeError: idx_to_component() missing 1 required positional argument: 'idx'

In [ ]:

    
def input_params(obj: IGH_Component):
    assert is_cls(obj, IGH_Component), "The component is not coming in as an IGH_Component"
    obj = convert_cls(obj, IGH_Component)
    parameters = []
    if obj.Params.Input:
        for p in obj.Params.Input:
                parameters.append(GHParam_(p))
    return parameters
def output_params(obj: IGH_Component):
    assert is_cls(obj, IGH_Component), "The component is not coming in as an IGH_Component"
    obj = convert_cls(obj, IGH_Component)
    parameters = []
    if obj.Params.Output:
        for p in obj.Params.Output:
            parameters.append(GHParam_(p))

    return parameters


        

In [ ]:
def src_rcpts(obj):
    inputs = []
    outputs = []
    param = is_cls(obj, IGH_Param)
    comp = convert_cls(obj, IGH_Component)
    # print(f"{obj.Name} -> ({comp},{param})")
    if comp:
        inputs = inputs + input_params(obj)
        outputs = outputs + output_params(obj)
    return inputs, outputs

for obj in issues:
    srcs, recps = src_rcpts(obj)
    print(f"{obj.Name} - s:{srcs} r:{recps}")    
    
       
            